In [24]:
# For Preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score,recall_score,precision_score
from dask.distributed import Client

In [25]:
X = pd.read_csv("application_train.csv")
df_test = pd.read_csv("application_test.csv")
X.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
X['CODE_GENDER'].replace('XNA',np.nan, inplace=True)
X['CODE_GENDER'].value_counts()

F    202448
M    105059
Name: CODE_GENDER, dtype: int64

In [27]:
X['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [28]:
X['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [32]:

le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in X:
    if X[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(X[col].unique())) <= 2:
            # Train on the training data
            le.fit(X[col])
            # Transform both training and testing data
            X[col] = le.transform(X[col])
           # df_test[col] = le.transform(df_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)


3 columns were label encoded.


In [33]:
# one-hot encoding of categorical variables
X = pd.get_dummies(X)
#df_test = pd.get_dummies(df_test)
list1=X.columns

print('Training Features shape: ', X.shape)
#print('Testing Features shape: ', df_test.shape)
print(list1)

Training Features shape:  (307511, 242)
Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       ...
       'HOUSETYPE_MODE_terraced house', 'WALLSMATERIAL_MODE_Block',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'EMERGENCYSTATE_MODE_No', 'EMERGENCYSTATE_MODE_Yes'],
      dtype='object', length=242)


In [34]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [35]:
# Missing values statistics
missing_values = missing_values_table(X)
mis_col=list(missing_values.index)

Your selected dataframe has 242 columns.
There are 62 columns that have missing values.


In [36]:

X.replace(np.nan,-999, inplace=True)
#X.astype('float32',inplace=True)
#print(newX[].describe())
y=X['TARGET']
X=X.drop('TARGET',axis=1)

In [37]:
# iterate below functions only

In [38]:
def split_train_test(X , y):
    return(train_test_split( X , y , test_size=0.2, random_state=42))

X_train, X_test, y_train, y_test =split_train_test(X,y)

In [39]:
def feature_eng(X_train,y_train,X_test,y_test):
    train_col=list(X_train.columns)
    test_col=list(X_test.columns)    
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X_train, y_train)
    model = SelectFromModel(clf, prefit=True)
    new_trainX = model.transform(X_train)
    n=np.shape(new_trainX)[1]
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]
    clist=[]
    for f in range(n):     
        clist.append(indices[f])
    new_train_col=[train_col[i] for i in clist]
    new_test_col=[test_col[i] for i in clist]
    X_train=pd.DataFrame(X_train)
    X_test=pd.DataFrame(X_test)
    y_train=pd.DataFrame(y_train)
    y_test=pd.DataFrame(y_test)
    df_new_train=pd.DataFrame(X_train.loc[:,new_train_col],columns=new_train_col)            
    df_new_test=pd.DataFrame(X_test.loc[:,new_test_col],columns=new_test_col)
    print("Improved number of features->",len(clist))    
    return(df_new_train,df_new_test)

df_new_train,df_new_test=feature_eng(X_train, y_train, X_test, y_test)

Improved number of features-> 82


In [40]:
def run_model_eval_metric(modelname,modelobj,X_train,Y_train,X_test,Y_test):
    # In order to let the models run faster we use dask dataframe
    
    modelobj.fit(X_train,Y_train)
    ypred_train=modelobj.predict(X_train)
    ypred_test=modelobj.predict(X_test)
    y_train_proba = modelobj.predict_proba(X_train)[:,1]
    y_test_proba = modelobj.predict_proba(X_test)[:,1]
    print('\n')
    print(modelname,"model:")
    confmat=confusion_matrix(Y_train,ypred_train)
    print("Train Confusion Matrix:")
    print(confmat)
    confmat=confusion_matrix(Y_test,ypred_test)
    print("Test Confusion Matrix:")
    print(confmat)
    metric=pd.DataFrame([[0]*5]*2,columns=['PRECISION','RECALL','F1_SCORE','ROC_AUC_SCORE','ACCURACY'],index=['Train','Test'],dtype='float32')
    metric['PRECISION']['Train']=round(precision_score(Y_train, ypred_train),3)
    metric['RECALL']['Train']=recall_score(Y_train, ypred_train)
    metric['F1_SCORE']['Train']=round(f1_score(Y_train, ypred_train),3)
    metric['ROC_AUC_SCORE']['Train']=roc_auc_score(Y_train,y_train_proba)
    metric['ACCURACY']['Train']=accuracy_score(Y_train,ypred_train)
    metric['PRECISION']['Test']=precision_score(Y_test, ypred_test)
    metric['RECALL']['Test']=recall_score(Y_test, ypred_test)
    metric['F1_SCORE']['Test']=f1_score(Y_test, ypred_test)
    metric['ROC_AUC_SCORE']['Test']=roc_auc_score(Y_test,y_test_proba)
    metric['ACCURACY']['Test']=accuracy_score(Y_test,ypred_test)
    print(metric)
   
    

In [41]:
def run_all_models(df_new_train,y_train,df_new_test,y_test):
    # K Nearest Neighbor
    knearest = KNeighborsClassifier(algorithm= 'auto', n_jobs= -1, n_neighbors= 1)
    run_model_eval_metric('K-Nearest Neighbor 1',knearest,df_new_train,y_train,df_new_test,y_test)
    
    knearest = KNeighborsClassifier(algorithm= 'auto', n_jobs= -1, n_neighbors= 3)
    run_model_eval_metric('K-Nearest Neighbor 3',knearest,df_new_train,y_train,df_new_test,y_test)

    # Random Forest
    randfor = RandomForestClassifier(n_estimators= 5, max_features= 'auto', max_depth= 60, bootstrap= True, n_jobs= -1)
    run_model_eval_metric('Random Forest',randfor,df_new_train,y_train,df_new_test,y_test)

    # Decision Trees
    dectree = DecisionTreeClassifier(random_state= 42, max_features= 'auto', max_depth= 40, criterion= 'gini')
    run_model_eval_metric('Decision Tree',dectree,df_new_train,y_train,df_new_test,y_test)

    # Logistic Regression
    logreg = LogisticRegression(penalty = 'l1', class_weight= {0: 0.1, 1: 0.9}, C= 0.5)
    run_model_eval_metric('Logistic Regression',logreg,df_new_train,y_train,df_new_test,y_test)

run_all_models(df_new_train,y_train,df_new_test,y_test)




K-Nearest Neighbor 1 model:
Train Confusion Matrix:
[[226132      0]
 [     0  19876]]
Test Confusion Matrix:
[[51975  4579]
 [ 4412   537]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.000000  1.000000        1.00000  1.000000
Test    0.104965  0.108507  0.106706        0.51377  0.853812


K-Nearest Neighbor 3 model:
Train Confusion Matrix:
[[224329   1803]
 [ 16083   3793]]
Test Confusion Matrix:
[[55357  1197]
 [ 4776   173]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   0.678000  0.190833  0.298000       0.933968  0.927295
Test    0.126277  0.034957  0.054755       0.528867  0.902883


Random Forest model:
Train Confusion Matrix:
[[225936    196]
 [  3630  16246]]
Test Confusion Matrix:
[[55734   820]
 [ 4726   223]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   0.988000  0.817368  0.895000       0.994287  0.984448
Test    0.213806  0.045060  0.074433       0.594956  0.909826


Decision Tree mode

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)




Logistic Regression model:
Train Confusion Matrix:
[[169804  56328]
 [ 10211   9665]]
Test Confusion Matrix:
[[42538 14016]
 [ 2490  2459]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   0.146000  0.486265  0.225000       0.670771  0.729525
Test    0.149256  0.496868  0.229556       0.675434  0.731623
